<a href="https://colab.research.google.com/github/Ryzxxl/SparkNLP--NLPTest/blob/main/BBC_NEWS_SparkNLP_NLPTest.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
! pip install -q pyspark==3.3.0 spark-nlp==4.3.2


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 281.3/281.3 MB 3.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 473.2/473.2 kB 10.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.7/199.7 kB 7.6 MB/s eta 0:00:00


In [2]:
import os
import sys

import sparknlp

from sparknlp.base import *
from sparknlp.common import *
from sparknlp.annotator import *

from pyspark.ml import Pipeline
from pyspark.sql import SparkSession

import pandas as pd
spark = sparknlp.start()

print("Spark NLP version: ", sparknlp.version())
print("Apache Spark version: ", spark.version)

spark

Spark NLP version:  4.3.2
Apache Spark version:  3.3.0


In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
import pandas as pd
data = pd.read_csv('/content/drive/MyDrive/data.csv') 

In [5]:
spark_df = spark.createDataFrame(data)

/usr/local/lib/python3.10/dist-packages/pyspark/sql/pandas/conversion.py:474: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  for column, series in pdf.iteritems():
/usr/local/lib/python3.10/dist-packages/pyspark/sql/pandas/conversion.py:486: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  for column, series in pdf.iteritems():


In [6]:
spark_df.show()

+---------+--------------------+-------------+
|ArticleId|                Text|     Category|
+---------+--------------------+-------------+
|     1833|worldcom ex-boss ...|     business|
|      154|german business c...|     business|
|     1101|bbc poll indicate...|     business|
|     1976|lifestyle  govern...|         tech|
|      917|enron bosses in $...|     business|
|     1582|howard  truanted ...|     politics|
|      651|wales silent on g...|        sport|
|     1797|french honour for...|entertainment|
|     2034|car giant hit by ...|     business|
|     1866|fockers fuel fest...|entertainment|
|     1683|blair rejects ira...|     politics|
|     1153|housewives lift c...|entertainment|
|     1028|uk coal plunges i...|     business|
|      812|bp surges ahead o...|     business|
|      707|ireland 21-19 arg...|        sport|
|     1588|wenger signs new ...|        sport|
|      342|u2 s desire to be...|entertainment|
|      486|hantuchova in dub...|        sport|
|     1344|me

In [ ]:
spark_df.take(2)

[Row(ArticleId=1833, Text='worldcom ex-boss launches defence lawyers defending former worldcom chief bernie ebbers against a battery of fraud charges have called a company whistleblower as their first witness.  cynthia cooper  worldcom s ex-head of internal accounting  alerted directors to irregular accounting practices at the us telecoms giant in 2002. her warnings led to the collapse of the firm following the discovery of an $11bn (£5.7bn) accounting fraud. mr ebbers has pleaded not guilty to charges of fraud and conspiracy.  prosecution lawyers have argued that mr ebbers orchestrated a series of accounting tricks at worldcom  ordering employees to hide expenses and inflate revenues to meet wall street earnings estimates. but ms cooper  who now runs her own consulting business  told a jury in new york on wednesday that external auditors arthur andersen had approved worldcom s accounting in early 2001 and 2002. she said andersen had given a  green light  to the procedures and practice

In [ ]:
from pyspark.sql.functions import col

spark_df.groupBy("Category") \
    .count() \
    .orderBy(col("count").desc()) \
    .show()

+-------------+-----+
|     Category|count|
+-------------+-----+
|        sport|  346|
|     business|  336|
|     politics|  274|
|entertainment|  273|
|         tech|  261|
+-------------+-----+



## Building Classification Pipeline
# LogReg with CountVectorizer

In [7]:
from pyspark.ml.feature import CountVectorizer, HashingTF, IDF, OneHotEncoder, StringIndexer, VectorAssembler, SQLTransformer


In [ ]:
%%time

document_assembler = DocumentAssembler() \
      .setInputCol("Text") \
      .setOutputCol("document")
    
tokenizer = Tokenizer() \
      .setInputCols(["document"]) \
      .setOutputCol("token")
      
normalizer = Normalizer() \
      .setInputCols(["token"]) \
      .setOutputCol("normalized")

stopwords_cleaner = StopWordsCleaner()\
      .setInputCols("normalized")\
      .setOutputCol("cleanTokens")\
      .setCaseSensitive(False)

stemmer = Stemmer() \
      .setInputCols(["cleanTokens"]) \
      .setOutputCol("stem")

finisher = Finisher() \
      .setInputCols(["stem"]) \
      .setOutputCols(["token_features"]) \
      .setOutputAsArray(True) \
      .setCleanAnnotations(False)

countVectors = CountVectorizer(inputCol="token_features", outputCol="features", vocabSize=10000, minDF=5)

label_stringIdx = StringIndexer(inputCol = "Category", outputCol = "label")

nlp_pipeline = Pipeline(
    stages=[document_assembler, 
            tokenizer,
            normalizer,
            stopwords_cleaner, 
            stemmer, 
            finisher,
            countVectors,
            label_stringIdx])

nlp_model = nlp_pipeline.fit(spark_df)

processed = nlp_model.transform(spark_df)

processed.count()


CPU times: user 345 ms, sys: 56.4 ms, total: 402 ms
Wall time: 28.8 s


1490

In [ ]:
processed.select('Text','token_features').show(truncate=50)

+--------------------------------------------------+--------------------------------------------------+
|                                              Text|                                    token_features|
+--------------------------------------------------+--------------------------------------------------+
|worldcom ex-boss launches defence lawyers defen...|[worldcom, exboss, launch, defenc, lawyer, defe...|
|german business confidence slides german busine...|[german, busi, confid, slide, german, busi, con...|
|bbc poll indicates economic gloom citizens in a...|[bbc, poll, indic, econom, gloom, citizen, majo...|
|lifestyle  governs mobile choice  faster  bette...|[lifestyl, govern, mobil, choic, faster, better...|
|enron bosses in $168m payout eighteen former en...|[enron, boss, m, payout, eighteen, former, enro...|
|howard  truanted to play snooker  conservative ...|[howard, truant, plai, snooker, conserv, leader...|
|wales silent on grand slam talk rhys williams s...|[wale, silen

In [ ]:
processed.select('token_features').take(2)


[Row(token_features=['worldcom', 'exboss', 'launch', 'defenc', 'lawyer', 'defend', 'former', 'worldcom', 'chief', 'berni', 'ebber', 'batteri', 'fraud', 'charg', 'call', 'compani', 'whistleblow', 'first', 'wit', 'cynthia', 'cooper', 'worldcom', 'exhead', 'intern', 'account', 'alert', 'director', 'irregular', 'account', 'practic', 'u', 'telecom', 'giant', 'warn', 'led', 'collaps', 'firm', 'follow', 'discoveri', 'bn', 'bn', 'account', 'fraud', 'mr', 'ebber', 'plead', 'guilti', 'charg', 'fraud', 'conspiraci', 'prosecut', 'lawyer', 'argu', 'mr', 'ebber', 'orchestr', 'seri', 'account', 'trick', 'worldcom', 'order', 'employe', 'hide', 'expens', 'inflat', 'revenu', 'meet', 'wall', 'street', 'earn', 'estim', 'm', 'cooper', 'run', 'consult', 'busi', 'told', 'juri', 'new', 'york', 'wednesdai', 'extern', 'auditor', 'arthur', 'andersen', 'approv', 'worldcom', 'account', 'earli', 'said', 'andersen', 'given', 'green', 'light', 'procedur', 'practic', 'us', 'worldcom', 'mr', 'ebber', 'lawyer', 'said', 

In [ ]:
processed.select('features').take(2)

[Row(features=SparseVector(5188, {0: 5.0, 1: 7.0, 2: 1.0, 3: 1.0, 4: 1.0, 5: 3.0, 7: 1.0, 10: 1.0, 12: 1.0, 13: 1.0, 26: 2.0, 27: 2.0, 32: 3.0, 37: 1.0, 50: 1.0, 68: 1.0, 97: 1.0, 110: 1.0, 112: 1.0, 114: 1.0, 115: 1.0, 116: 1.0, 117: 2.0, 124: 1.0, 135: 1.0, 147: 2.0, 148: 1.0, 152: 1.0, 154: 1.0, 159: 3.0, 163: 3.0, 219: 1.0, 243: 1.0, 256: 3.0, 280: 1.0, 294: 1.0, 305: 7.0, 306: 1.0, 310: 1.0, 314: 2.0, 357: 1.0, 377: 1.0, 379: 1.0, 418: 1.0, 461: 1.0, 521: 1.0, 565: 1.0, 569: 1.0, 588: 2.0, 596: 1.0, 597: 1.0, 609: 1.0, 624: 1.0, 637: 1.0, 639: 1.0, 642: 1.0, 643: 1.0, 663: 1.0, 725: 1.0, 739: 4.0, 807: 1.0, 825: 1.0, 844: 1.0, 851: 1.0, 861: 1.0, 914: 1.0, 920: 1.0, 930: 1.0, 932: 1.0, 958: 1.0, 961: 1.0, 974: 3.0, 984: 1.0, 1050: 1.0, 1071: 1.0, 1078: 1.0, 1109: 1.0, 1111: 1.0, 1209: 2.0, 1232: 2.0, 1234: 9.0, 1243: 2.0, 1288: 1.0, 1299: 1.0, 1310: 2.0, 1315: 1.0, 1372: 1.0, 1404: 4.0, 1405: 7.0, 1495: 1.0, 1498: 1.0, 1622: 1.0, 1667: 2.0, 1679: 1.0, 1739: 2.0, 1740: 1.0, 1968: 1

In [ ]:
processed.select('Text','features','label').show()

+--------------------+--------------------+-----+
|                Text|            features|label|
+--------------------+--------------------+-----+
|worldcom ex-boss ...|(5188,[0,1,2,3,4,...|  1.0|
|german business c...|(5188,[0,2,12,13,...|  1.0|
|bbc poll indicate...|(5188,[0,2,4,5,6,...|  1.0|
|lifestyle  govern...|(5188,[0,2,3,4,6,...|  4.0|
|enron bosses in $...|(5188,[0,2,3,5,7,...|  1.0|
|howard  truanted ...|(5188,[0,1,4,5,9,...|  2.0|
|wales silent on g...|(5188,[0,9,11,12,...|  0.0|
|french honour for...|(5188,[0,1,4,7,8,...|  3.0|
|car giant hit by ...|(5188,[0,2,3,5,7,...|  1.0|
|fockers fuel fest...|(5188,[0,2,3,4,5,...|  3.0|
|blair rejects ira...|(5188,[0,1,3,8,10...|  2.0|
|housewives lift c...|(5188,[2,7,8,12,1...|  3.0|
|uk coal plunges i...|(5188,[0,1,2,3,5,...|  1.0|
|bp surges ahead o...|(5188,[0,2,3,5,12...|  1.0|
|ireland 21-19 arg...|(5188,[8,9,11,13,...|  0.0|
|wenger signs new ...|(5188,[0,2,3,4,5,...|  0.0|
|u2 s desire to be...|(5188,[0,1,2,3,4,...|  3.0|


In [ ]:
(trainingData, testData) = processed.randomSplit([0.75, 0.25], seed = 100)
print("Training Dataset Count: " + str(trainingData.count()))
print("Test Dataset Count: " + str(testData.count()))

Training Dataset Count: 1118
Test Dataset Count: 372


In [ ]:
trainingData.printSchema()

root
 |-- ArticleId: long (nullable = true)
 |-- Text: string (nullable = true)
 |-- Category: string (nullable = true)
 |-- document: array (nullable = true)
 |    |-- element: struct (containsNull = true)
 |    |    |-- annotatorType: string (nullable = true)
 |    |    |-- begin: integer (nullable = false)
 |    |    |-- end: integer (nullable = false)
 |    |    |-- result: string (nullable = true)
 |    |    |-- metadata: map (nullable = true)
 |    |    |    |-- key: string
 |    |    |    |-- value: string (valueContainsNull = true)
 |    |    |-- embeddings: array (nullable = true)
 |    |    |    |-- element: float (containsNull = false)
 |-- token: array (nullable = true)
 |    |-- element: struct (containsNull = true)
 |    |    |-- annotatorType: string (nullable = true)
 |    |    |-- begin: integer (nullable = false)
 |    |    |-- end: integer (nullable = false)
 |    |    |-- result: string (nullable = true)
 |    |    |-- metadata: map (nullable = true)
 |    |    |   

In [ ]:
from pyspark.ml.classification import LogisticRegression

lr = LogisticRegression(regParam=0.1, maxIter=100, elasticNetParam=0.0, family="multinomial")

lrModel = lr.fit(trainingData)

predictions = lrModel.transform(testData)

predictions.filter(predictions['prediction'] == 0) \
    .select("Text","Category","probability","label","prediction") \
    .orderBy("probability", ascending=False) \
    .show(n = 10, truncate = 30)

+------------------------------+--------+------------------------------+-----+----------+
|                          Text|Category|                   probability|label|prediction|
+------------------------------+--------+------------------------------+-----+----------+
|holmes back on form in birm...|   sport|[0.9999992841027223,1.96724...|  0.0|       0.0|
|federer joins all-time grea...|   sport|[0.9999858149757638,2.50011...|  0.0|       0.0|
|ireland surge past scots ir...|   sport|[0.9999525939790581,7.62550...|  0.0|       0.0|
|bridge could miss rest of s...|   sport|[0.9998728016892534,1.91619...|  0.0|       0.0|
|blackburn v burnley ewood p...|   sport|[0.999813916186194,3.389904...|  0.0|       0.0|
|chelsea hold arsenal a grip...|   sport|[0.9997745493449134,6.11558...|  0.0|       0.0|
|preview: ireland v england ...|   sport|[0.9996801798038802,5.75293...|  0.0|       0.0|
|wenger keeping faith with a...|   sport|[0.9996431846417368,3.73588...|  0.0|       0.0|
|a year to

In [ ]:
from pyspark.ml.evaluation import MulticlassClassificationEvaluator

evaluator = MulticlassClassificationEvaluator(predictionCol="prediction")

evaluator.evaluate(predictions)

0.9650001255469579

In [ ]:
from sklearn.metrics import confusion_matrix, classification_report, accuracy_score
y_true = predictions.select("label")
y_true = y_true.toPandas()

y_pred = predictions.select("prediction")
y_pred = y_pred.toPandas()

In [ ]:
y_pred.prediction.value_counts()

0.0    97
1.0    87
4.0    66
3.0    63
2.0    59
Name: prediction, dtype: int64

In [ ]:
cnf_matrix = confusion_matrix(list(y_true.label.astype(int)), list(y_pred.prediction.astype(int)))
cnf_matrix

array([[96,  0,  1,  0,  0],
       [ 1, 82,  1,  0,  0],
       [ 0,  4, 57,  1,  0],
       [ 0,  0,  0, 59,  1],
       [ 0,  1,  0,  3, 65]])

In [ ]:
print(classification_report(y_true.label, y_pred.prediction))
print(accuracy_score(y_true.label, y_pred.prediction))

              precision    recall  f1-score   support

         0.0       0.99      0.99      0.99        97
         1.0       0.94      0.98      0.96        84
         2.0       0.97      0.92      0.94        62
         3.0       0.94      0.98      0.96        60
         4.0       0.98      0.94      0.96        69

    accuracy                           0.97       372
   macro avg       0.96      0.96      0.96       372
weighted avg       0.97      0.97      0.97       372

0.9650537634408602


In [ ]:
from pyspark.ml.feature import HashingTF, IDF

hashingTF = HashingTF(inputCol="token_features", outputCol="rawFeatures", numFeatures=10000)

idf = IDF(inputCol="rawFeatures", outputCol="features", minDocFreq=5) #minDocFreq: remove sparse terms

nlp_pipeline_tf = Pipeline(
    stages=[document_assembler, 
            tokenizer,
            normalizer,
            stopwords_cleaner, 
            stemmer, 
            finisher,
            hashingTF,
            idf,
            label_stringIdx])

nlp_model_tf = nlp_pipeline_tf.fit(spark_df)

processed_tf = nlp_model_tf.transform(spark_df)

processed_tf.count()

1490

In [ ]:
processed_tf.select('Text','features','label').show()

+--------------------+--------------------+-----+
|                Text|            features|label|
+--------------------+--------------------+-----+
|worldcom ex-boss ...|(10000,[7,63,221,...|  1.0|
|german business c...|(10000,[24,70,203...|  1.0|
|bbc poll indicate...|(10000,[8,140,320...|  1.0|
|lifestyle  govern...|(10000,[7,29,70,1...|  4.0|
|enron bosses in $...|(10000,[15,33,63,...|  1.0|
|howard  truanted ...|(10000,[89,221,26...|  2.0|
|wales silent on g...|(10000,[21,23,154...|  0.0|
|french honour for...|(10000,[312,398,5...|  3.0|
|car giant hit by ...|(10000,[14,116,15...|  1.0|
|fockers fuel fest...|(10000,[328,488,5...|  3.0|
|blair rejects ira...|(10000,[23,70,206...|  2.0|
|housewives lift c...|(10000,[144,253,2...|  3.0|
|uk coal plunges i...|(10000,[63,117,29...|  1.0|
|bp surges ahead o...|(10000,[63,114,15...|  1.0|
|ireland 21-19 arg...|(10000,[30,116,26...|  0.0|
|wenger signs new ...|(10000,[157,288,4...|  0.0|
|u2 s desire to be...|(10000,[7,15,29,3...|  3.0|


In [ ]:
(trainingData, testData) = processed_tf.randomSplit([0.75, 0.25], seed = 100)
print("Training Dataset Count: " + str(trainingData.count()))
print("Test Dataset Count: " + str(testData.count()))


Training Dataset Count: 1118
Test Dataset Count: 372


In [ ]:
lr2 = LogisticRegression(regParam=0.1, maxIter=100, elasticNetParam=0.0, family="multinomial")

lrModel_tf = lr2.fit(trainingData)

predictions_tf = lrModel_tf.transform(testData)

predictions_tf.select("Text","Category","probability","label","prediction") \
    .orderBy("probability", ascending=False) \
    .show(n = 10, truncate = 30)

+------------------------------+--------+------------------------------+-----+----------+
|                          Text|Category|                   probability|label|prediction|
+------------------------------+--------+------------------------------+-----+----------+
|holmes back on form in birm...|   sport|[0.9999948703345454,3.41985...|  0.0|       0.0|
|federer joins all-time grea...|   sport|[0.9999754262884323,4.60097...|  0.0|       0.0|
|bridge could miss rest of s...|   sport|[0.9997298472999848,4.02940...|  0.0|       0.0|
|llewellyn plans wales retir...|   sport|[0.9995557041948206,3.01338...|  0.0|       0.0|
|ireland surge past scots ir...|   sport|[0.9994834138231586,6.46792...|  0.0|       0.0|
|a year to remember for iris...|   sport|[0.9994205804598082,4.91829...|  0.0|       0.0|
|blackburn v burnley ewood p...|   sport|[0.9993749079711313,1.58398...|  0.0|       0.0|
|chelsea hold arsenal a grip...|   sport|[0.9990608499919214,1.36338...|  0.0|       0.0|
|wenger ke

In [ ]:
y_true = predictions_tf.select("label")
y_true = y_true.toPandas()

y_pred = predictions_tf.select("prediction")
y_pred = y_pred.toPandas()

print(classification_report(y_true.label, y_pred.prediction))
print(accuracy_score(y_true.label, y_pred.prediction))

              precision    recall  f1-score   support

         0.0       0.98      1.00      0.99        97
         1.0       0.95      0.98      0.96        84
         2.0       0.98      0.94      0.96        62
         3.0       0.94      0.97      0.95        60
         4.0       0.98      0.94      0.96        69

    accuracy                           0.97       372
   macro avg       0.97      0.96      0.97       372
weighted avg       0.97      0.97      0.97       372

0.967741935483871


## LogReg with Spark NLP Glove Word Embeddings

In [ ]:
document_assembler = DocumentAssembler() \
      .setInputCol("Text") \
      .setOutputCol("document")
    
tokenizer = Tokenizer() \
      .setInputCols(["document"]) \
      .setOutputCol("token")
    
normalizer = Normalizer() \
      .setInputCols(["token"]) \
      .setOutputCol("normalized")

stopwords_cleaner = StopWordsCleaner()\
      .setInputCols("normalized")\
      .setOutputCol("cleanTokens")\
      .setCaseSensitive(False)

glove_embeddings = WordEmbeddingsModel().pretrained() \
      .setInputCols(["document",'cleanTokens'])\
      .setOutputCol("embeddings")\
      .setCaseSensitive(False)

embeddingsSentence = SentenceEmbeddings() \
      .setInputCols(["document", "embeddings"]) \
      .setOutputCol("sentence_embeddings") \
      .setPoolingStrategy("AVERAGE")
    
embeddings_finisher = EmbeddingsFinisher() \
      .setInputCols(["sentence_embeddings"]) \
      .setOutputCols(["finished_sentence_embeddings"]) \
      .setOutputAsVector(True)\
      .setCleanAnnotations(False)

explodeVectors = SQLTransformer(statement=
      "SELECT EXPLODE(finished_sentence_embeddings) AS features, * FROM __THIS__")

label_stringIdx = StringIndexer(inputCol = "Category", outputCol = "label")

nlp_pipeline_w2v = Pipeline(
    stages=[document_assembler, 
            tokenizer,
            normalizer,
            stopwords_cleaner, 
            glove_embeddings,
            embeddingsSentence,
            embeddings_finisher,
            explodeVectors,
            label_stringIdx])

nlp_model_w2v = nlp_pipeline_w2v.fit(spark_df)

processed_w2v = nlp_model_w2v.transform(spark_df)

processed_w2v.count()

glove_100d download started this may take some time.
Approximate size to download 145.3 MB
[OK!]


1490

In [ ]:
processed_w2v.columns

['features',
 'ArticleId',
 'Text',
 'Category',
 'document',
 'token',
 'normalized',
 'cleanTokens',
 'embeddings',
 'sentence_embeddings',
 'finished_sentence_embeddings',
 'label']

In [ ]:
processed_w2v.select('cleanTokens').take(1)


[Row(cleanTokens=[Row(annotatorType='token', begin=0, end=7, result='worldcom', metadata={'sentence': '0'}, embeddings=[]), Row(annotatorType='token', begin=9, end=14, result='exboss', metadata={'sentence': '0'}, embeddings=[]), Row(annotatorType='token', begin=17, end=24, result='launches', metadata={'sentence': '0'}, embeddings=[]), Row(annotatorType='token', begin=26, end=32, result='defence', metadata={'sentence': '0'}, embeddings=[]), Row(annotatorType='token', begin=34, end=40, result='lawyers', metadata={'sentence': '0'}, embeddings=[]), Row(annotatorType='token', begin=42, end=50, result='defending', metadata={'sentence': '0'}, embeddings=[]), Row(annotatorType='token', begin=52, end=57, result='former', metadata={'sentence': '0'}, embeddings=[]), Row(annotatorType='token', begin=59, end=66, result='worldcom', metadata={'sentence': '0'}, embeddings=[]), Row(annotatorType='token', begin=68, end=72, result='chief', metadata={'sentence': '0'}, embeddings=[]), Row(annotatorType='to

In [ ]:
processed_w2v.select('finished_sentence_embeddings').take(1)


[Row(finished_sentence_embeddings=[DenseVector([0.1437, -0.1435, 0.1251, -0.122, 0.0234, -0.0685, -0.0403, 0.0208, -0.2078, -0.0395, 0.2774, 0.1897, 0.0593, 0.1102, -0.1828, -0.2921, 0.106, 0.0407, -0.3182, 0.1434, -0.046, -0.209, 0.1107, 0.1091, -0.1284, -0.1339, 0.1091, -0.1376, -0.2712, -0.0369, 0.3489, 0.317, -0.2112, -0.2237, -0.1914, 0.1335, 0.0601, 0.0699, 0.1367, 0.0045, -0.3001, -0.1923, 0.2107, -0.1359, -0.1833, 0.0642, 0.0434, -0.3502, 0.0429, -0.5357, 0.2422, -0.1395, 0.1654, 0.5328, -0.1822, -1.3902, -0.1226, -0.1377, 1.138, 0.412, -0.1213, 0.1091, -0.1132, 0.0361, 0.3306, -0.3115, 0.3167, 0.4455, 0.2343, 0.1333, -0.1088, -0.0144, -0.3547, -0.2582, 0.0342, -0.2139, -0.0035, -0.0054, -0.7143, -0.0038, 0.7572, -0.0728, -0.0391, 0.0223, -0.9537, -0.3432, 0.0132, -0.0042, -0.043, -0.2652, 0.1228, -0.171, -0.1259, -0.0848, 0.1928, 0.2324, 0.0154, -0.0958, 0.2338, 0.0395])])]

In [ ]:
processed_w2v.select("features").take(1)


[Row(features=DenseVector([0.1437, -0.1435, 0.1251, -0.122, 0.0234, -0.0685, -0.0403, 0.0208, -0.2078, -0.0395, 0.2774, 0.1897, 0.0593, 0.1102, -0.1828, -0.2921, 0.106, 0.0407, -0.3182, 0.1434, -0.046, -0.209, 0.1107, 0.1091, -0.1284, -0.1339, 0.1091, -0.1376, -0.2712, -0.0369, 0.3489, 0.317, -0.2112, -0.2237, -0.1914, 0.1335, 0.0601, 0.0699, 0.1367, 0.0045, -0.3001, -0.1923, 0.2107, -0.1359, -0.1833, 0.0642, 0.0434, -0.3502, 0.0429, -0.5357, 0.2422, -0.1395, 0.1654, 0.5328, -0.1822, -1.3902, -0.1226, -0.1377, 1.138, 0.412, -0.1213, 0.1091, -0.1132, 0.0361, 0.3306, -0.3115, 0.3167, 0.4455, 0.2343, 0.1333, -0.1088, -0.0144, -0.3547, -0.2582, 0.0342, -0.2139, -0.0035, -0.0054, -0.7143, -0.0038, 0.7572, -0.0728, -0.0391, 0.0223, -0.9537, -0.3432, 0.0132, -0.0042, -0.043, -0.2652, 0.1228, -0.171, -0.1259, -0.0848, 0.1928, 0.2324, 0.0154, -0.0958, 0.2338, 0.0395]))]

In [ ]:
processed_w2v.select('Text','features','label').show()


+--------------------+--------------------+-----+
|                Text|            features|label|
+--------------------+--------------------+-----+
|worldcom ex-boss ...|[0.14365777373313...|  1.0|
|german business c...|[0.04885630682110...|  1.0|
|bbc poll indicate...|[-0.0909639447927...|  1.0|
|lifestyle  govern...|[-0.1047650575637...|  4.0|
|enron bosses in $...|[0.25210666656494...|  1.0|
|howard  truanted ...|[-0.0044781574979...|  2.0|
|wales silent on g...|[-0.1021427735686...|  0.0|
|french honour for...|[0.03469001874327...|  3.0|
|car giant hit by ...|[0.12876743078231...|  1.0|
|fockers fuel fest...|[0.01742288284003...|  3.0|
|blair rejects ira...|[-0.1817228943109...|  2.0|
|housewives lift c...|[0.12551599740982...|  3.0|
|uk coal plunges i...|[-0.0331566482782...|  1.0|
|bp surges ahead o...|[0.09505154192447...|  1.0|
|ireland 21-19 arg...|[-0.0510397553443...|  0.0|
|wenger signs new ...|[0.10450825840234...|  0.0|
|u2 s desire to be...|[-0.0528848618268...|  3.0|


In [ ]:
(trainingData, testData) = processed_w2v.randomSplit([0.75, 0.25], seed = 100)
print("Training Dataset Count: " + str(trainingData.count()))
print("Test Dataset Count: " + str(testData.count()))

Training Dataset Count: 1118
Test Dataset Count: 372


In [ ]:
## By removing such rows, we can optimize the performance of our machine learning algorithms, as we can skip the computation for those rows where the features vector is all zeros.
from pyspark.sql.functions import udf

@udf("long")
def num_nonzeros(v):
    return v.numNonzeros()

testData = testData.where(num_nonzeros("features") != 0)

In [ ]:
lrModel_w2v = lr.fit(trainingData)

In [ ]:
predictions_w2v = lrModel_w2v.transform(testData)

predictions_w2v.select("Text","Category","probability","label","prediction") \
    .orderBy("probability", ascending=False) \
    .show(n = 10, truncate = 30)

+------------------------------+--------+------------------------------+-----+----------+
|                          Text|Category|                   probability|label|prediction|
+------------------------------+--------+------------------------------+-----+----------+
|benitez issues warning to g...|   sport|[0.999354628677034,1.305838...|  0.0|       0.0|
|gerrard plays down european...|   sport|[0.9977441826873454,8.25964...|  0.0|       0.0|
|cudicini misses carling cup...|   sport|[0.9974225551750332,5.30672...|  0.0|       0.0|
|wilkinson return  unlikely ...|   sport|[0.9969352814994883,5.32838...|  0.0|       0.0|
|england given tough sevens ...|   sport|[0.9963728653818645,0.00172...|  0.0|       0.0|
|slovakia reach hopman cup f...|   sport|[0.9960847622257419,3.83563...|  0.0|       0.0|
|sydney return for henin-har...|   sport|[0.9949767349285513,0.00108...|  0.0|       0.0|
|desailly backs blues reveng...|   sport|[0.994739305804544,0.002234...|  0.0|       0.0|
|benitez d

In [ ]:
from sklearn.metrics import confusion_matrix, classification_report, accuracy_score
import pandas as pd

y_true = predictions_w2v.select("label")
y_true = y_true.toPandas()

y_pred = predictions_w2v.select("prediction")
y_pred = y_pred.toPandas()

print(classification_report(y_true.label, y_pred.prediction))
print(accuracy_score(y_true.label, y_pred.prediction))

              precision    recall  f1-score   support

         0.0       0.99      1.00      0.99        79
         1.0       0.93      0.94      0.94        87
         2.0       0.96      0.93      0.94        69
         3.0       0.98      1.00      0.99        59
         4.0       0.97      0.96      0.97        78

    accuracy                           0.97       372
   macro avg       0.97      0.97      0.97       372
weighted avg       0.97      0.97      0.96       372

0.9650537634408602


In [ ]:
processed_w2v.select('Text','cleanTokens.result').show(truncate=50)


+--------------------------------------------------+--------------------------------------------------+
|                                              Text|                                            result|
+--------------------------------------------------+--------------------------------------------------+
|worldcom ex-boss launches defence lawyers defen...|[worldcom, exboss, launches, defence, lawyers, ...|
|german business confidence slides german busine...|[german, business, confidence, slides, german, ...|
|bbc poll indicates economic gloom citizens in a...|[bbc, poll, indicates, economic, gloom, citizen...|
|lifestyle  governs mobile choice  faster  bette...|[lifestyle, governs, mobile, choice, faster, be...|
|enron bosses in $168m payout eighteen former en...|[enron, bosses, m, payout, eighteen, former, en...|
|howard  truanted to play snooker  conservative ...|[howard, truanted, play, snooker, conservative,...|
|wales silent on grand slam talk rhys williams s...|[wales, sile

## LogReg with Spark NLP Bert Embeddings


In [ ]:
document_assembler = DocumentAssembler() \
      .setInputCol("Text") \
      .setOutputCol("document")
    
tokenizer = Tokenizer() \
      .setInputCols(["document"]) \
      .setOutputCol("token")
    
normalizer = Normalizer() \
      .setInputCols(["token"]) \
      .setOutputCol("normalized")

stopwords_cleaner = StopWordsCleaner()\
      .setInputCols("normalized")\
      .setOutputCol("cleanTokens")\
      .setCaseSensitive(False)

bert_embeddings = BertEmbeddings.pretrained('bert_base_cased', 'en') \
      .setInputCols(["document",'cleanTokens'])\
      .setOutputCol("bert")\
      .setCaseSensitive(False)\

embeddingsSentence = SentenceEmbeddings() \
      .setInputCols(["document", "bert"]) \
      .setOutputCol("sentence_embeddings") \
      .setPoolingStrategy("AVERAGE")
    
embeddings_finisher = EmbeddingsFinisher() \
      .setInputCols(["sentence_embeddings"]) \
      .setOutputCols(["finished_sentence_embeddings"]) \
      .setOutputAsVector(True)\
      .setCleanAnnotations(False)

label_stringIdx = StringIndexer(inputCol = "Category", outputCol = "label")

nlp_pipeline_bert = Pipeline(
    stages=[document_assembler, 
            tokenizer,
            normalizer,
            stopwords_cleaner, 
            bert_embeddings,
            embeddingsSentence,
            embeddings_finisher,
            label_stringIdx])
nlp_model_bert = nlp_pipeline_bert.fit(spark_df)
processed_bert = nlp_model_bert.transform(spark_df)


bert_base_cased download started this may take some time.
Approximate size to download 389.1 MB
[OK!]


In [ ]:
from pyspark.sql.functions import explode

processed_bert = processed_bert.withColumn("features", explode(processed_bert.finished_sentence_embeddings))

processed_bert.select('Text','features','label').show()

+--------------------+--------------------+-----+
|                Text|            features|label|
+--------------------+--------------------+-----+
|worldcom ex-boss ...|[0.17285257577896...|  1.0|
|german business c...|[-0.1185487955808...|  1.0|
|bbc poll indicate...|[-0.1185369566082...|  1.0|
|lifestyle  govern...|[0.19791017472743...|  4.0|
|enron bosses in $...|[9.24773456063121...|  1.0|
|howard  truanted ...|[0.18108294904232...|  2.0|
|wales silent on g...|[0.02165658958256...|  0.0|
|french honour for...|[0.05820161849260...|  3.0|
|car giant hit by ...|[0.18970946967601...|  1.0|
|fockers fuel fest...|[0.02853241935372...|  3.0|
|blair rejects ira...|[0.00927552208304...|  2.0|
|housewives lift c...|[0.09813518077135...|  3.0|
|uk coal plunges i...|[0.02478078566491...|  1.0|
|bp surges ahead o...|[-0.0316127724945...|  1.0|
|ireland 21-19 arg...|[0.11042474955320...|  0.0|
|wenger signs new ...|[-0.0729556754231...|  0.0|
|u2 s desire to be...|[0.00726967584341...|  3.0|


In [ ]:
# set seed for reproducibility
(trainingData, testData) = processed_bert.randomSplit([0.75, 0.25], seed = 100)

# print("Training Dataset Count: " + str(trainingData.count()))
# print("Test Dataset Count: " + str(testData.count()))

In [ ]:
from pyspark.ml.classification import LogisticRegression

lr = LogisticRegression(regParam=0.1, maxIter=100, elasticNetParam=0.0, family="multinomial")

lrModel = lr.fit(trainingData)

In [ ]:
from pyspark.sql.functions import udf

@udf("long")
def num_nonzeros(v):
    return v.numNonzeros()

testData = testData.where(num_nonzeros("features") != 0)

In [ ]:
predictions = lrModel.transform(testData)

predictions.select("Text","category","probability","label","prediction") \
    .orderBy("probability", ascending=False) \
    .show(n = 10, truncate = 30)

+------------------------------+--------+------------------------------+-----+----------+
|                          Text|category|                   probability|label|prediction|
+------------------------------+--------+------------------------------+-----+----------+
|stevens named in england li...|   sport|[0.9992222137802227,4.02181...|  0.0|       0.0|
|hodgson shoulders england b...|   sport|[0.9990033886768118,3.57715...|  0.0|       0.0|
|benitez deflects blame from...|   sport|[0.9987627531098847,4.01127...|  0.0|       0.0|
|barcelona title hopes hit b...|   sport|[0.998441711498892,7.708451...|  0.0|       0.0|
|ferrero eyes return to top ...|   sport|[0.9981155879744107,5.75670...|  0.0|       0.0|
|ferguson rues failure to cu...|   sport|[0.9981094680320829,4.40219...|  0.0|       0.0|
|federer breezes into semi-f...|   sport|[0.9980688978023965,4.51704...|  0.0|       0.0|
|liverpool pledge to keep ge...|   sport|[0.9980453847539553,2.87808...|  0.0|       0.0|
|williams 

In [ ]:
from sklearn.metrics import confusion_matrix, classification_report, accuracy_score
import pandas as pd

In [ ]:
df = predictions.select('Text','category','label','prediction').toPandas()


In [ ]:
print(classification_report(df.label, df.prediction))
print(accuracy_score(df.label, df.prediction))

              precision    recall  f1-score   support

         0.0       0.99      0.98      0.98        97
         1.0       0.90      0.93      0.91        84
         2.0       0.92      0.90      0.91        62
         3.0       0.93      0.95      0.94        60
         4.0       0.96      0.93      0.94        69

    accuracy                           0.94       372
   macro avg       0.94      0.94      0.94       372
weighted avg       0.94      0.94      0.94       372

0.9408602150537635


# ElmoEmbeddings + LogReg

In [ ]:
document_assembler = DocumentAssembler() \
      .setInputCol("Text") \
      .setOutputCol("document")
    
tokenizer = Tokenizer() \
      .setInputCols(["document"]) \
      .setOutputCol("token")
    
normalizer = Normalizer() \
      .setInputCols(["token"]) \
      .setOutputCol("normalized")

stopwords_cleaner = StopWordsCleaner()\
      .setInputCols("normalized")\
      .setOutputCol("cleanTokens")\
      .setCaseSensitive(False)

elmo_embeddings = ElmoEmbeddings.pretrained()\
      .setPoolingLayer("word_emb")\
      .setInputCols(["document",'cleanTokens'])\
      .setOutputCol("elmo")

embeddingsSentence = SentenceEmbeddings() \
      .setInputCols(["document", "elmo"]) \
      .setOutputCol("sentence_embeddings") \
      .setPoolingStrategy("AVERAGE")
    
embeddings_finisher = EmbeddingsFinisher() \
      .setInputCols(["sentence_embeddings"]) \
      .setOutputCols(["finished_sentence_embeddings"]) \
      .setOutputAsVector(True)\
      .setCleanAnnotations(False)

label_stringIdx = StringIndexer(inputCol = "Category", outputCol = "label")


nlp_pipeline_elmo = Pipeline(
    stages=[document_assembler, 
            tokenizer,
            normalizer,
            stopwords_cleaner, 
            elmo_embeddings,
            embeddingsSentence,
            embeddings_finisher,
            label_stringIdx])

nlp_model_elmo = nlp_pipeline_elmo.fit(spark_df)
processed_elmo = nlp_model_elmo.transform(spark_df)
processed_elmo.count()

elmo download started this may take some time.
Approximate size to download 334.1 MB
[OK!]


1490

In [9]:
(trainingData, testData) = spark_df.randomSplit([0.7, 0.3], seed = 100)


In [ ]:
processed_trainingData = nlp_model_elmo.transform(trainingData)

In [ ]:
processed_trainingData.count()


1053

In [ ]:
processed_testData = nlp_model_elmo.transform(testData)

processed_testData.count()

437

In [ ]:
processed_trainingData.columns


['ArticleId',
 'Text',
 'Category',
 'document',
 'token',
 'normalized',
 'cleanTokens',
 'elmo',
 'sentence_embeddings',
 'finished_sentence_embeddings',
 'label']

In [ ]:
processed_trainingData.select('cleanTokens', 'document', 'elmo', 'sentence_embeddings').show(truncate = 50)


+--------------------------------------------------+--------------------------------------------------+--------------------------------------------------+--------------------------------------------------+
|                                       cleanTokens|                                          document|                                              elmo|                               sentence_embeddings|
+--------------------------------------------------+--------------------------------------------------+--------------------------------------------------+--------------------------------------------------+
|[{token, 0, 7, worldcom, {sentence -> 0}, []}, ...|[{document, 0, 1841, worldcom boss  left books ...|[{word_embeddings, 0, 7, worldcom, {isOOV -> fa...|[{sentence_embeddings, 0, 1841, worldcom boss  ...|
|[{token, 0, 5, berlin, {sentence -> 0}, []}, {t...|[{document, 0, 1818, berlin cheers for anti-naz...|[{word_embeddings, 0, 5, berlin, {isOOV -> fals...|[{sentence_embeddings,

In [ ]:
from pyspark.sql.functions import explode

processed_testData= processed_testData.withColumn("features", explode(processed_testData.finished_sentence_embeddings))

processed_trainingData= processed_trainingData.withColumn("features", explode(processed_trainingData.finished_sentence_embeddings))

In [ ]:
from pyspark.sql.functions import udf

@udf("long")
def num_nonzeros(v):
    return v.numNonzeros()

processed_testData = processed_testData.where(num_nonzeros("features") != 0)

In [ ]:
from pyspark.ml.classification import LogisticRegression

lr = LogisticRegression(maxIter=20, regParam=0.3, elasticNetParam=0)

lrModel = lr.fit(processed_trainingData)

In [ ]:
predictions = lrModel.transform(processed_testData)

predictions.select("Text","category","probability","label","prediction") \
    .orderBy("probability", ascending=False) \
    .show(10, truncate = 30)

+------------------------------+--------+------------------------------+-----+----------+
|                          Text|category|                   probability|label|prediction|
+------------------------------+--------+------------------------------+-----+----------+
|benitez issues warning to g...|   sport|[0.9968600008576672,1.51449...|  0.0|       0.0|
|hodgson shoulders england b...|   sport|[0.9947468416520254,0.00103...|  0.0|       0.0|
|barcelona title hopes hit b...|   sport|[0.994580509495302,0.001386...|  0.0|       0.0|
|england claim dubai sevens ...|   sport|[0.9939821734152283,0.00146...|  0.0|       0.0|
|benitez deflects blame from...|   sport|[0.9939744658722092,0.00119...|  0.0|       0.0|
|hearts of oak 3-2 cotonspor...|   sport|[0.9925993854893356,0.00111...|  0.0|       0.0|
|robinson answers critics en...|   sport|[0.9917527157668187,0.00125...|  0.0|       0.0|
|injury sidelines philippous...|   sport|[0.9916379483207169,9.90806...|  0.0|       0.0|
|bridge co

In [ ]:
from sklearn.metrics import classification_report, accuracy_score

df = predictions.select('Text','category','label','prediction').toPandas()
print(classification_report(df.label, df.prediction))
print(accuracy_score(df.label, df.prediction))

              precision    recall  f1-score   support

         0.0       1.00      0.99      1.00       113
         1.0       0.94      0.96      0.95        97
         2.0       0.93      0.96      0.95        74
         3.0       0.94      0.94      0.94        72
         4.0       0.96      0.93      0.94        81

    accuracy                           0.96       437
   macro avg       0.96      0.96      0.96       437
weighted avg       0.96      0.96      0.96       437

0.9588100686498856


In [ ]:
df

,Text,category,label,prediction
0,s korean credit card firm rescued south korea ...,business,1.0,1.0
1,gamers snap up new sony psp gamers have bought...,tech,4.0,4.0
2,row threatens hendrix museum plan proposals to...,entertainment,3.0,2.0
3,federer claims dubai crown world number one ro...,sport,0.0,0.0
4,telegraph newspapers axe 90 jobs the daily and...,business,1.0,2.0
...,...,...,...,...
367,jones files conte lawsuit marion jones has fil...,sport,0.0,0.0
368,lopez misses uk charity premiere jennifer lope...,entertainment,3.0,3.0
369,progress on new internet domains by early 2005...,tech,4.0,4.0
370,davies favours gloucester future wales hooker ...,sport,0.0,0.0


# ClassifierDL + Glove + Preprocess


In [ ]:
document = DocumentAssembler()\
    .setInputCol("Text")\
    .setOutputCol("document")

tokenizer = Tokenizer() \
      .setInputCols(["document"]) \
      .setOutputCol("token")

lemma = LemmatizerModel.pretrained('lemma_antbnc') \
      .setInputCols(["token"]) \
      .setOutputCol("lemma")
      
glove_embeddings = WordEmbeddingsModel().pretrained() \
      .setInputCols(["document",'lemma'])\
      .setOutputCol("embeddings")\
      .setCaseSensitive(False)

lemma_pipeline = Pipeline(
    stages=[document, 
            tokenizer,
            lemma,
            glove_embeddings])

lemma_antbnc download started this may take some time.
Approximate size to download 907.6 KB
[OK!]
glove_100d download started this may take some time.
Approximate size to download 145.3 MB
[OK!]


In [ ]:
(trainingData, testData) = spark_df.randomSplit([0.75, 0.25], seed = 100)

lemma_pipeline.fit(trainingData).transform(trainingData).show(truncate=30)


+---------+------------------------------+-------------+------------------------------+------------------------------+------------------------------+------------------------------+
|ArticleId|                          Text|     Category|                      document|                         token|                         lemma|                    embeddings|
+---------+------------------------------+-------------+------------------------------+------------------------------+------------------------------+------------------------------+
|        2|worldcom boss  left books a...|     business|[{document, 0, 1841, worldc...|[{token, 0, 7, worldcom, {s...|[{token, 0, 7, worldcom, {s...|[{word_embeddings, 0, 7, wo...|
|       11|berlin cheers for anti-nazi...|entertainment|[{document, 0, 1818, berlin...|[{token, 0, 5, berlin, {sen...|[{token, 0, 5, berlin, {sen...|[{word_embeddings, 0, 5, be...|
|       15|moya emotional after davis ...|        sport|[{document, 0, 2410, moya e...|[{token,

In [ ]:
document_assembler = DocumentAssembler() \
      .setInputCol("Text") \
      .setOutputCol("document")
    
tokenizer = Tokenizer() \
      .setInputCols(["document"]) \
      .setOutputCol("token")
    
normalizer = Normalizer() \
      .setInputCols(["token"]) \
      .setOutputCol("normalized")

stopwords_cleaner = StopWordsCleaner()\
      .setInputCols("normalized")\
      .setOutputCol("cleanTokens")\
      .setCaseSensitive(False)

lemma = LemmatizerModel.pretrained('lemma_antbnc') \
      .setInputCols(["cleanTokens"]) \
      .setOutputCol("lemma")

glove_embeddings = WordEmbeddingsModel().pretrained() \
      .setInputCols(["document",'lemma'])\
      .setOutputCol("embeddings")\
      .setCaseSensitive(False)

embeddingsSentence = SentenceEmbeddings() \
      .setInputCols(["document", "embeddings"]) \
      .setOutputCol("sentence_embeddings") \
      .setPoolingStrategy("AVERAGE")

classsifierdl = ClassifierDLApproach()\
      .setInputCols(["sentence_embeddings"])\
      .setOutputCol("class")\
      .setLabelColumn("Category")\
      .setMaxEpochs(10)\
      .setEnableOutputLogs(True)

clf_pipeline = Pipeline(
    stages=[document_assembler, 
            tokenizer,
            normalizer,
            stopwords_cleaner, 
            lemma, 
            glove_embeddings,
            embeddingsSentence,
            classsifierdl])

lemma_antbnc download started this may take some time.
Approximate size to download 907.6 KB
[OK!]
glove_100d download started this may take some time.
Approximate size to download 145.3 MB
[OK!]


In [ ]:
!rm -rf classifier_dl_pipeline_glove


In [ ]:
#clf_pipeline.save('/content/drive/MyDrive/John Snow Lbs/classifier_dl_pipeline_glove')

In [ ]:
clf_pipelineModel = clf_pipeline.fit(trainingData)

In [ ]:
df = clf_pipelineModel.transform(testData).select('Category','Text',"class.result").toPandas()
df.head()

,Category,Text,result
0,business,s korean credit card firm rescued south korea ...,[business]
1,tech,gamers snap up new sony psp gamers have bought...,[tech]
2,entertainment,row threatens hendrix museum plan proposals to...,[politics]
3,sport,federer claims dubai crown world number one ro...,[sport]
4,business,telegraph newspapers axe 90 jobs the daily and...,[tech]


In [ ]:
df

,Category,Text,result
0,business,s korean credit card firm rescued south korea ...,[business]
1,tech,gamers snap up new sony psp gamers have bought...,[tech]
2,entertainment,row threatens hendrix museum plan proposals to...,[politics]
3,sport,federer claims dubai crown world number one ro...,[sport]
4,business,telegraph newspapers axe 90 jobs the daily and...,[tech]
...,...,...,...
367,sport,jones files conte lawsuit marion jones has fil...,[politics]
368,entertainment,lopez misses uk charity premiere jennifer lope...,[entertainment]
369,tech,progress on new internet domains by early 2005...,[tech]
370,sport,davies favours gloucester future wales hooker ...,[sport]


In [ ]:
df['result'] = df['result'].apply(lambda x: x[0])

In [ ]:
df

,Category,Text,result
0,business,s korean credit card firm rescued south korea ...,business
1,tech,gamers snap up new sony psp gamers have bought...,tech
2,entertainment,row threatens hendrix museum plan proposals to...,politics
3,sport,federer claims dubai crown world number one ro...,sport
4,business,telegraph newspapers axe 90 jobs the daily and...,tech
...,...,...,...
367,sport,jones files conte lawsuit marion jones has fil...,politics
368,entertainment,lopez misses uk charity premiere jennifer lope...,entertainment
369,tech,progress on new internet domains by early 2005...,tech
370,sport,davies favours gloucester future wales hooker ...,sport


In [ ]:
from sklearn.metrics import classification_report, accuracy_score

print(classification_report(df.Category, df.result))
print(accuracy_score(df.Category, df.result))

               precision    recall  f1-score   support

     business       0.94      0.93      0.93        84
entertainment       0.93      0.92      0.92        60
     politics       0.85      0.90      0.88        62
        sport       1.00      0.97      0.98        97
         tech       0.91      0.93      0.92        69

     accuracy                           0.93       372
    macro avg       0.93      0.93      0.93       372
 weighted avg       0.93      0.93      0.93       372

0.9327956989247311


# sent_bert_use_cmlm_en_base Model


In [15]:
document = DocumentAssembler()\
    .setInputCol("Text")\
    .setOutputCol("document")

bert_cmlm = BertSentenceEmbeddings.pretrained('sent_bert_use_cmlm_en_base', 'en')\
    .setInputCols(["document"])\
    .setOutputCol("sentence_embeddings")

classsifierdl = ClassifierDLApproach()\
    .setInputCols(["sentence_embeddings"])\
    .setOutputCol("class")\
    .setLabelColumn("Category")\
    .setMaxEpochs(10)\
    .setEnableOutputLogs(True)\
    .setLr(0.001)

bert_cmlm_clf_pipeline = Pipeline(stages = [document,
                                            bert_cmlm,
                                            classsifierdl])

sent_bert_use_cmlm_en_base download started this may take some time.
Approximate size to download 391.6 MB
[OK!]


In [ ]:
! rm -r /root/annotator_logs


rm: cannot remove '/root/annotator_logs': No such file or directory


In [18]:
(trainingData, testDataset) = spark_df.randomSplit([0.75, 0.25], seed = 100)

In [20]:
bert_cmlm_pipelineModel = bert_cmlm_clf_pipeline.fit(trainingData)

In [ ]:
from sklearn.metrics import classification_report

preds = bert_cmlm_pipelineModel.transform(testDataset)

In [ ]:
preds_df = preds.select('Category','Text',"class.result").toPandas()

preds_df['result'] = preds_df['result'].apply(lambda x : x[0])


In [ ]:
print (classification_report(preds_df['Category'], preds_df['result']))


               precision    recall  f1-score   support

     business       0.95      0.98      0.96        84
entertainment       1.00      0.97      0.98        60
     politics       0.97      0.97      0.97        62
        sport       1.00      0.98      0.99        97
         tech       0.96      0.99      0.97        69

     accuracy                           0.98       372
    macro avg       0.98      0.98      0.98       372
 weighted avg       0.98      0.98      0.98       372



# NLP TEST

In [ ]:
!pip install nlptest


In [11]:
!pip install johnsnowlabs


  Created wheel for pyspark: filename=pyspark-3.1.2-py2.py3-none-any.whl size=212880770 sha256=ac98da2b09f25b52150898f47f59bc862af572b418338e0ee01af9f325bdd9d7
  Stored in directory: /root/.cache/pip/wheels/ef/70/50/7882e1bcb5693225f7cc86698f10953201b48b3f36317c2d18
  Created wheel for databricks-cli: filename=databricks_cli-0.17.6-py3-none-any.whl size=143237 sha256=60f22efa80c03719f291a1309f2d2e64dc734de3992eee0c6ff75f2e4f14bc45
  Stored in directory: /root/.cache/pip/wheels/7b/85/a1/37b9b24f55c1da8ea4795765de41713b8a2cccad89c866e26b
Successfully built pyspark databricks-cli
  Attempting uninstall: spark-nlp
    Found existing installation: spark-nlp 4.3.2
    Uninstalling spark-nlp-4.3.2:
      Successfully uninstalled spark-nlp-4.3.2
  Attempting uninstall: py4j
    Found existing installation: py4j 0.10.9.5
    Uninstalling py4j-0.10.9.5:
      Successfully uninstalled py4j-0.10.9.5
  Attempting uninstall: pyspark
    Found existing installation: pyspark 3.3.0
    Uninstalling pys

In [ ]:
trainDataset = trainDataset.toPandas()

In [ ]:
trainDataset = trainDataset.rename({'Category':'label'}, axis = 1)

In [ ]:
trainDataset

,ArticleId,Text,label
0,2,worldcom boss left books alone former worldc...,business
1,11,berlin cheers for anti-nazi film a german movi...,entertainment
2,15,moya emotional after davis cup win carlos moya...,sport
3,18,connors boost for british tennis former world ...,sport
4,19,japanese banking battle at an end japan s sumi...,business
...,...,...,...
1113,2201,uk pioneers digital film network the world s f...,tech
1114,2206,dance music not dead says fatboy dj norman coo...,entertainment
1115,2207,kennedy questions trust of blair lib dem leade...,politics
1116,2208,california sets fines for spyware the makers o...,tech


In [ ]:
trainDataset.to_csv('/content/drive/MyDrive/John Snow Lbs/train.csv')

In [12]:
traindata = '/content/drive/MyDrive/John Snow Lbs/train.csv'

In [21]:
from nlptest import Harness
harness = Harness(task="text-classification", model=bert_cmlm_pipelineModel, data=traindata)


In [22]:
harness.configure(
{'defaults': {'min_pass_rate': 0.65},
 'tests': {'robustness': {'lowercase': {'min_pass_rate': 0.60}, 
                          'uppercase':{'min_pass_rate': 0.60},
                          'add_typo':{'min_pass_rate': 0.90}},
                          # 'add_contraction':{'min_pass_rate': 0.60},
                          # 'add_punctuation':{'min_pass_rate': 0.60},
                          # 'titlecase':{'min_pass_rate': 0.60}
          }
 }
 )

{'defaults': {'min_pass_rate': 0.65},
 'tests': {'robustness': {'lowercase': {'min_pass_rate': 0.6},
   'uppercase': {'min_pass_rate': 0.6},
   'add_typo': {'min_pass_rate': 0.9}}}}

In [24]:
harness.generate()

Generating testcases... (robustness): 100%|██████████| 1/1 [00:00<00:00,  5.92it/s]


In [25]:
harness.testcases()

,category,test_type,original,test_case,expected_result
0,robustness,lowercase,worldcom boss left books alone former worldc...,worldcom boss left books alone former worldc...,business
1,robustness,lowercase,berlin cheers for anti-nazi film a german movi...,berlin cheers for anti-nazi film a german movi...,entertainment
2,robustness,lowercase,moya emotional after davis cup win carlos moya...,moya emotional after davis cup win carlos moya...,politics
3,robustness,lowercase,connors boost for british tennis former world ...,connors boost for british tennis former world ...,business
4,robustness,lowercase,japanese banking battle at an end japan s sumi...,japanese banking battle at an end japan s sumi...,politics
...,...,...,...,...,...
3349,robustness,add_typo,uk pioneers digital film network the world s f...,uk pioneers digital film network the world s f...,business
3350,robustness,add_typo,dance music not dead says fatboy dj norman coo...,dance music not dead says fatboy dj norman coo...,business
3351,robustness,add_typo,kennedy questions trust of blair lib dem leade...,kennedy questions trust of blair lib dem leade...,entertainment
3352,robustness,add_typo,california sets fines for spyware the makers o...,california sets fines for spyware the makers o...,politics


In [26]:
save_dir = '/content/drive/MyDrive/John Snow Lbs/Harness_Test_Case'
harness.save(save_dir)


In [27]:
harness.run()


Running testcases... : 100%|██████████| 3354/3354 [1:28:38<00:00,  1.59s/it]


In [28]:
harness.report()

,category,test_type,fail_count,pass_count,pass_rate,minimum_pass_rate,pass
0,robustness,lowercase,0,1118,100%,60%,True
1,robustness,uppercase,0,1118,100%,60%,True
2,robustness,add_typo,134,984,88%,90%,False


In [29]:
harness.generated_results()

,category,test_type,original,test_case,expected_result,actual_result,pass
0,robustness,lowercase,worldcom boss left books alone former worldc...,worldcom boss left books alone former worldc...,business,business,True
1,robustness,lowercase,berlin cheers for anti-nazi film a german movi...,berlin cheers for anti-nazi film a german movi...,entertainment,entertainment,True
2,robustness,lowercase,moya emotional after davis cup win carlos moya...,moya emotional after davis cup win carlos moya...,politics,politics,True
3,robustness,lowercase,connors boost for british tennis former world ...,connors boost for british tennis former world ...,business,business,True
4,robustness,lowercase,japanese banking battle at an end japan s sumi...,japanese banking battle at an end japan s sumi...,politics,politics,True
...,...,...,...,...,...,...,...
3349,robustness,add_typo,uk pioneers digital film network the world s f...,uk pioneers digital film network the world s f...,business,business,True
3350,robustness,add_typo,dance music not dead says fatboy dj norman coo...,dance music not dead says fatboy dj norman coo...,business,business,True
3351,robustness,add_typo,kennedy questions trust of blair lib dem leade...,kennedy questions trust of blair lib dem leade...,entertainment,entertainment,True
3352,robustness,add_typo,california sets fines for spyware the makers o...,california sets fines for spyware the makers o...,politics,business,False


In [30]:
harness.augment("/content/drive/MyDrive/John Snow Lbs/train.csv", "/content/drive/MyDrive/John Snow Lbs/augmented_train.csv", inplace=False)

In [31]:
import pandas as pd
trainDataset_aug = pd.read_csv('/content/drive/MyDrive/John Snow Lbs/augmented_train.csv', header=None)

In [33]:

column_names = ['Text', 'Category']
trainDataset_aug.columns = column_names

In [34]:
trainDataset_aug

,Text,Category
0,worldcom boss left books alone former worldc...,b
1,berlin cheers for anti-nazi film a german movi...,e
2,moya emotional after davis cup win carlos moya...,s
3,connors boost for british tennis former world ...,s
4,japanese banking battle at an end japan s sumi...,b
...,...,...
1672,satellite mapping aids darfur relief aid worke...,t
1673,stormy year for property insurers a string of ...,b
1674,royal couple watch nation s mood prince charle...,p
1675,standard life concern at lse bid standard life...,b


In [35]:
mapping_dict = {'s': 'sport', 't': 'tech', 'b': 'business', 'e': 'entertainment', 'p': 'politics'}
trainDataset_aug['Category'] = trainDataset_aug['Category'].replace(mapping_dict)

In [36]:
trainDataset_aug


,Text,Category
0,worldcom boss left books alone former worldc...,business
1,berlin cheers for anti-nazi film a german movi...,entertainment
2,moya emotional after davis cup win carlos moya...,sport
3,connors boost for british tennis former world ...,sport
4,japanese banking battle at an end japan s sumi...,business
...,...,...
1672,satellite mapping aids darfur relief aid worke...,tech
1673,stormy year for property insurers a string of ...,business
1674,royal couple watch nation s mood prince charle...,politics
1675,standard life concern at lse bid standard life...,business


In [37]:
trainDataset_aug = spark.createDataFrame(trainDataset_aug)
trainDataset_aug.show()

+--------------------+-------------+
|                Text|     Category|
+--------------------+-------------+
|worldcom boss  le...|     business|
|berlin cheers for...|entertainment|
|moya emotional af...|        sport|
|connors boost for...|        sport|
|japanese banking ...|     business|
|mobile audio ente...|         tech|
|moya fights back ...|        sport|
|royal couple watc...|     politics|
|firefox browser t...|         tech|
|debate needed  on...|     politics|
|german growth goe...|     business|
|franz man seeks g...|entertainment|
|jamieson issues w...|     politics|
|england children ...|     politics|
|stock market eyes...|     business|
|standard life cut...|     business|
|court rejects $28...|     business|
|india power share...|     business|
|microsoft gets th...|         tech|
|tech helps disabl...|         tech|
+--------------------+-------------+
only showing top 20 rows



In [38]:
bert_cmlm_pipelineModel_aug = bert_cmlm_clf_pipeline.fit(trainDataset_aug)

In [39]:
from sklearn.metrics import classification_report

preds = bert_cmlm_pipelineModel.transform(testDataset)

In [53]:
preds_df = preds.select('Category','Text',"class.result").toPandas()


In [54]:
preds_df['result'] = preds_df['result'].apply(lambda x : x[0])

In [55]:
preds_df

,Category,Text,result
0,business,s korean credit card firm rescued south korea ...,business
1,tech,gamers snap up new sony psp gamers have bought...,tech
2,entertainment,row threatens hendrix museum plan proposals to...,entertainment
3,sport,federer claims dubai crown world number one ro...,sport
4,business,telegraph newspapers axe 90 jobs the daily and...,business
...,...,...,...
367,sport,jones files conte lawsuit marion jones has fil...,sport
368,entertainment,lopez misses uk charity premiere jennifer lope...,entertainment
369,tech,progress on new internet domains by early 2005...,tech
370,sport,davies favours gloucester future wales hooker ...,sport


In [56]:
print(classification_report(preds_df['Category'], preds_df['result']))

               precision    recall  f1-score   support

     business       0.95      0.98      0.96        84
entertainment       1.00      0.98      0.99        60
     politics       0.97      0.97      0.97        62
        sport       1.00      0.98      0.99        97
         tech       0.97      0.99      0.98        69

     accuracy                           0.98       372
    macro avg       0.98      0.98      0.98       372
 weighted avg       0.98      0.98      0.98       372

